In [ ]:
using_colab = True

In [ ]:
if using_colab:
    import torch
    import torchvision
    print("PyTorch version:", torch.__version__)
    print("Torchvision version:", torchvision.__version__)
    print("CUDA is available:", torch.cuda.is_available())
    import sys
    !{sys.executable} -m pip install opencv-python matplotlib
    !{sys.executable} -m pip install 'git+https://github.com/facebookresearch/segment-anything.git'

    !mkdir images
    !wget -P images https://raw.githubusercontent.com/facebookresearch/segment-anything/main/notebooks/images/dog.jpg
    !wget https://dl.fbaipublicfiles.com/segment_anything/sam_vit_l_0b3195.pth

PyTorch version: 2.2.1+cu121
Torchvision version: 0.17.1+cu121
CUDA is available: True
  Cloning https://github.com/facebookresearch/segment-anything.git to /tmp/pip-req-build-ztxuf2ah
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/segment-anything.git /tmp/pip-req-build-ztxuf2ah
  Resolved https://github.com/facebookresearch/segment-anything.git to commit 6fdee8f2727f4506cfbbe553e23b895e27956588
  Preparing metadata (setup.py) ... done
  Created wheel for segment-anything: filename=segment_anything-1.0-py3-none-any.whl size=36590 sha256=ce39b3dd0e242231546e0b1307cf964ded86fecfa16904ed33b831022a51e270
  Stored in directory: /tmp/pip-ephem-wheel-cache-fxu237s8/wheels/10/cf/59/9ccb2f0a1bcc81d4fbd0e501680b5d088d690c6cfbc02dc99d
Successfully built segment-anything
--2024-04-22 09:28:51--  https://raw.githubusercontent.com/facebookresearch/segment-anything/main/notebooks/images/dog.jpg
Resolving raw.githubusercontent.com (raw.githubusercontent.com

In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import pandas as pd
from skimage import io
import torch
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator

In [ ]:
def filter_masks_by_size(masks, image, max_size):
    """
    Filters masks based on their size to remove very small and very large masks.
    min_size and max_size should be defined based on the domain knowledge about the grain sizes.
    """
    filtered_masks = []
    total_pixels = image.shape[0] * image.shape[1]
    for mask in masks:
        mask_size = np.sum(mask['segmentation'])
        if mask_size <= max_size:
            filtered_masks.append(mask)
    return filtered_masks

def calculate_circularity(mask):
    area = np.sum(mask)
    contours, _ = cv2.findContours(mask.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if contours:
        perimeter = cv2.arcLength(contours[0], True)
        if perimeter == 0:
            return 0
        circularity = 4 * np.pi * (area / (perimeter ** 2))
    else:
        circularity = 0
    return circularity

def average_intensity(image, mask):
    masked_image = cv2.bitwise_and(image, image, mask=mask.astype(np.uint8))
    return np.mean(masked_image[mask > 0])

def calculate_iou(mask1, mask2):
    intersection = np.logical_and(mask1, mask2)
    union = np.logical_or(mask1, mask2)
    iou = np.sum(intersection) / np.sum(union)
    return iou

def apply_kmeans(masks, image):
    avg_intensities = [average_intensity(image, mask['segmentation']) for mask in masks]
    features = np.array(avg_intensities).reshape(-1, 1)
    kmeans = KMeans(n_clusters=2, random_state=0).fit(features)
    labels = kmeans.labels_
    cluster_centers = kmeans.cluster_centers_
    darker_cluster_index = 0 if cluster_centers[0] < cluster_centers[1] else 1
    darker_masks = [masks[i] for i in range(len(masks)) if labels[i] == darker_cluster_index]
    lighter_masks = [masks[i] for i in range(len(masks)) if labels[i] != darker_cluster_index]
    return darker_masks,lighter_masks
def show_anns(anns):
    if len(anns) == 0:
        return
    sorted_anns = sorted(anns, key=(lambda x: x['area']), reverse=True)
    ax = plt.gca()
    ax.set_autoscale_on(False)

    img = np.ones((sorted_anns[0]['segmentation'].shape[0], sorted_anns[0]['segmentation'].shape[1], 4))
    img[:,:,3] = 0
    for ann in sorted_anns:
        m = ann['segmentation']
        color_mask = np.concatenate([np.random.random(3), [0.35]])
        img[m] = color_mask
    ax.imshow(img)


def show_masks(image, masks, title,output_path):
    fig, ax = plt.subplots(figsize=(20, 20))
    ax.imshow(image)
    show_anns(masks)
    plt.title(title)
    ax.axis('off')
    plt.savefig(output_path, bbox_inches='tight', pad_inches=0)
    plt.close()

def resolve_overlaps(masks):
    keep_masks = []
    n = len(masks)
    removed = set()

    for i in range(n):
        if i in removed:
            continue
        current_mask = masks[i]['segmentation']
        max_area = np.sum(current_mask)
        best_mask = masks[i]

        for j in range(i + 1, n):
            if j in removed:
                continue
            compare_mask = masks[j]['segmentation']
            if calculate_iou(current_mask, compare_mask) > 0.5:  # Threshold for considering an overlap
                if np.sum(compare_mask) > max_area:
                    max_area = np.sum(compare_mask)
                    removed.add(i)
                    best_mask = masks[j]
                    current_mask = compare_mask
                else:
                    removed.add(j)

        keep_masks.append(best_mask)

    return keep_masks

def calculate_coverage_percentage(image, masks):
    total_pixels = image.shape[0] * image.shape[1]  # Total number of pixels in the image
    mask_area_sum = sum(np.sum(mask['segmentation']) for mask in masks)  # Sum the areas of all masks
    coverage_percentage = (mask_area_sum / total_pixels) * 100  # Convert to percentage
    return coverage_percentage

In [ ]:
def process_directory(image_directory, output_directory, coverage_csv):
    device = torch.device("cuda")
    sam_checkpoint = "sam_vit_l_0b3195.pth"
    model_type = "vit_l"
    sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
    sam.to(device=device)
    mask_generator = SamAutomaticMaskGenerator(sam)
    mask_generator_2 = SamAutomaticMaskGenerator(
        model=sam,
        points_per_side=52,
        pred_iou_thresh=0.7,
        stability_score_thresh=0.92,
        crop_n_layers=1,
        crop_n_points_downscale_factor=2,
        min_mask_region_area=60
    )
    coverage_data = []

    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

    for filename in os.listdir(image_directory):
        if filename.lower().endswith(".jpg"):
            file_path = os.path.join(image_directory, filename)
            image = cv2.imread(file_path)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            masks = mask_generator.generate(image)
            masks2 = mask_generator_2.generate(image)

            darker_masks1, lighter_masks1 = apply_kmeans(masks, image)
            darker_masks2, lighter_masks2 = apply_kmeans(masks2, image)

            # Combine and resolve overlaps for darker masks
            combined_darker_masks = darker_masks1 + darker_masks2
            #non_overlapping_darker = resolve_overlaps(combined_darker_masks)
            output_path = os.path.join(output_directory, f"{filename[:-4]}_darker.jpg")
            show_masks(image, combined_darker_masks,"Darker Masks",  output_path)

            coverage_percentage = calculate_coverage_percentage(image,combined_darker_masks)


            # Calculate and display properties for lighter masks
            combined_lighter_masks = lighter_masks1 + lighter_masks2
            #non_overlapping_lighter = resolve_overlaps(combined_lighter_masks)
            max_size = 0.1 * image.shape[0] * image.shape[1]   # Example: 10% of the image area
            filtered_lighter_masks = filter_masks_by_size(combined_lighter_masks, image, max_size)
            output_path = os.path.join(output_directory, f"{filename[:-4]}_lighter.jpg")
            show_masks(image, filtered_lighter_masks,"Lighter Masks",output_path)

            # Calculate average size and circularity for lighter masks
            average_size = np.mean([np.sum(mask['segmentation']) for mask in filtered_lighter_masks])
            average_circularity = np.mean([calculate_circularity(mask['segmentation']) for mask in filtered_lighter_masks])
            coverage_data.append([filename, coverage_percentage, average_size, average_circularity])

    df = pd.DataFrame(coverage_data, columns=['Filename', 'Coverage Percentage','Average Size', 'Average Circularity'])
    df.to_csv(coverage_csv, index=False)

In [ ]:
from google.colab import drive
# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Set your directories and CSV file path
image_directory = '//content/drive/MyDrive/HOLE_DET_REM2'
#image_directory_2 = '/content/drive/MyDrive/NO_HOLE_HOLEDET'
output_directory = '/content/drive/MyDrive/Processed_Images_v3'
#output_directory_2 = '/content/drive/MyDrive/Processed_Images_nh'
coverage_csv = '/content/drive/MyDrive/coverage_results2.csv'
#coverage_csv_2 = '/content/drive/MyDrive/coverage_results_nh.csv'
# Execute processing
process_directory(image_directory, output_directory, coverage_csv)
#process_directory(image_directory_2, output_directory_2, coverage_csv_2)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:8